In [4]:
from datasets import load_dataset
import pandas as pd

In [48]:
drug_metadata = pd.DataFrame(load_dataset("tahoebio/Tahoe-100M","drug_metadata", split="train"))
drug_metadata = drug_metadata.loc[drug_metadata["targets"].notna()]
target_to_drug = dict(zip(drug_metadata["targets"], drug_metadata["drug"]))
cell_metadata = pd.DataFrame(load_dataset("tahoebio/Tahoe-100M","cell_line_metadata", split="train"))
depmap = pd.read_csv("CRISPRGeneDependency.csv", index_col=0)

In [49]:
depmap_to_tahoe = dict(zip(cell_metadata["Cell_ID_DepMap"], cell_metadata["Cell_ID_Cellosaur"]))
depmap.index = depmap.index.map(depmap_to_tahoe)
depmap = depmap.loc[depmap.index.notna()]
depmap.columns = depmap.columns.map(lambda x: x.split(" ")[0])

In [50]:
depmap = depmap.reset_index().melt(id_vars='index')
depmap["drug"] = depmap["variable"].map(target_to_drug)

In [53]:
depmap = depmap.dropna(subset=["drug"])
depmap["drug_cellline"] = depmap["drug"] + "_" + depmap["index"]

/tmp/ipykernel_162928/1821687993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap["drug_cellline"] = depmap["drug"] + "_" + depmap["index"]


In [56]:
depmap.index = depmap["drug_cellline"]
depmap["depmap_dependency_score"] = depmap["value"]
depmap[["depmap_dependency_score"]].to_csv("clin-oracle-tahoe-hack-2025/data_for_classifier/depmap_dependency_scores.tsv", index=True, sep='\t')

/tmp/ipykernel_162928/792430270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap["depmap_dependency_score"] = depmap["value"]
